<a href="https://colab.research.google.com/github/icehelix72/TSPredEML/blob/main/Monthly_Rason_Data_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Achmed Gerland


In [ ]:
#Install terlebih dahulu library yang dibutuhkan
!pip install siphon datetime python-dateutil pandas requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 9.4 MB/s eta 0:00:00


In [ ]:
#Lakukan mount drive terlebih dahulu untuk menyimpan file spreadsheet di google drive
#Jalankan skrip ini sekali saja
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Impor library yang akan digunakan
from datetime import datetime, timedelta
from siphon.simplewebservice.wyoming import WyomingUpperAir
from dateutil.relativedelta import relativedelta
import pandas as pd
from requests import HTTPError
import sys
import time

#Fungsi untuk membuat list tanggal dan waktu dengan interval jam tertentu pada suatu bulan
def generate_dates(year, month, interval_hours):
    start_date = datetime(year, month, 1)
    end_date = start_date + relativedelta(months=1)

    current_date = start_date
    dates = []

    while current_date < end_date:
        dates.append(current_date)
        current_date += timedelta(hours=interval_hours)

    return dates

In [ ]:
year = int(input("Tahun: ")) #Mengambil masukan tahun sebagai integer
month = int(input("Bulan (dalam angka 1-12): ")) #Mengambil masukan bulan sebagai integer
dates = generate_dates(year, month, 12) #Membuat list tanggal dan waktu
station_icao = "WIII" #input("Kode ICAO Stasiun (lihat di web Wyoming): ") #Mengambil masukan kode ICAO stasiun yang hendak dicari
#Kode ICAO tidak sama dengan kode stasiun yang berupa angka. Untuk lebih jelasnya, silahkan lihat kode ICAO yang ada di web Wyoming

Tahun: 2023
Bulan (dalam angka 1-12): 12


In [ ]:
import time

max_retries = 10  # Set the maximum number of retries
retry_delay = 30 # Set the delay in seconds before retrying

retries = 0
while retries < max_retries:
    try:
        print(f"Attempt {retries + 1} of {max_retries}...")

        # Membuat dictionary untuk memasukkan dataframe dari data radiosonde 12 jam-an
        rason_data = {}
        http_error_occurred_in_attempt = False # Flag to track if HTTPError occurs in this attempt

        # Mengambil data radiosonde dari library siphon berdasaran waktu dan stasiun
        for date in dates:
            print("Memproses data tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
            try:
                rason_data[date.strftime(r'%Y-%m-%d %HZ')] = df = WyomingUpperAir.request_data(date, station_icao)
            except KeyError:
                print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
            except ValueError:
                print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
            except IndexError:
                print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
            except HTTPError:
                print("HTTPError: Server sedang sibuk, mencoba lagi...")
                http_error_occurred_in_attempt = True # Set flag
                break # Break the inner for loop to trigger a retry attempt

        # Mengecek apakah HTTPError terjadi pada percobaan ini.
        # Jika tidak, keluar dari retry loop (terlepas dari apakah semua data terkumpul).
        if not http_error_occurred_in_attempt:
            print("No HTTPError occurred in this attempt.")
            break # Exit the retry loop if no HTTPError was encountered

    except Exception as e:
        # This catches any other exceptions during the process that were not caught inside the inner loop.
        print(f"An unexpected error occurred: {e}")
        # You might want to decide if other exceptions should also trigger a retry or stop the program.
        # For now, they will increment retries and potentially retry.

    # Increment retries and sleep if an HTTPError occurred or another unexpected exception occurred
    retries += 1
    if retries < max_retries and http_error_occurred_in_attempt: # Only sleep and retry if HTTPError occurred
        print(f"Retrying in {retry_delay} seconds...")
        time.sleep(retry_delay)
    elif retries < max_retries and not http_error_occurred_in_attempt:
         # If no HTTPError and not all data collected, maybe you still want to retry for other errors?
         # Or just stop? The current logic stops if no HTTPError occurred.
         pass # No sleep if we are breaking the loop anyway

# After the loop
if not http_error_occurred_in_attempt:
    print("Process completed without HTTP errors.")
    if len(rason_data.items()) == len(dates):
        print("Data collection completed successfully.")
    else:
        print("Data collection was incomplete (possibly due to other errors like KeyError).")
        collected_dates = [datetime.strptime(d_str, r'%Y-%m-%d %HZ') for d_str in rason_data.keys()]
        missing_dates = [d for d in dates if d not in collected_dates]
        print(f"Missing data for dates: {[d.strftime(r'%Y-%m-%d %HZ') for d in missing_dates]}")
else:
     # This part is reached if max_retries were reached due to persistent HTTP errors
     print("Maximum retries reached due to persistent HTTP errors. Data collection was incomplete.")

Attempt 1 of 10...
Memproses data tanggal 2023-12-01 00Z
Memproses data tanggal 2023-12-01 12Z
Memproses data tanggal 2023-12-02 00Z
Memproses data tanggal 2023-12-02 12Z
Memproses data tanggal 2023-12-03 00Z
Memproses data tanggal 2023-12-03 12Z
Memproses data tanggal 2023-12-04 00Z
Memproses data tanggal 2023-12-04 12Z
Memproses data tanggal 2023-12-05 00Z
Memproses data tanggal 2023-12-05 12Z
Memproses data tanggal 2023-12-06 00Z
Memproses data tanggal 2023-12-06 12Z
Memproses data tanggal 2023-12-07 00Z
Memproses data tanggal 2023-12-07 12Z
Memproses data tanggal 2023-12-08 00Z
Memproses data tanggal 2023-12-08 12Z
Memproses data tanggal 2023-12-09 00Z
Memproses data tanggal 2023-12-09 12Z
Memproses data tanggal 2023-12-10 00Z
Memproses data tanggal 2023-12-10 12Z
Memproses data tanggal 2023-12-11 00Z
Memproses data tanggal 2023-12-11 12Z
Memproses data tanggal 2023-12-12 00Z
Memproses data tanggal 2023-12-12 12Z
Memproses data tanggal 2023-12-13 00Z
Memproses data tanggal 2023-12-

In [ ]:
#Untuk mengeluarkan file data yang tanggalnya dipisahkan per-sheet
#Keluarkan file spreadsheet yang berisi semua dataframe yang ada di dictionary
with pd.ExcelWriter(r"/content/drive/MyDrive/Skripsi/WIII/"+str(year)+"/"+station_icao+str(month).zfill(2)+str(year)+".xlsx") as writer:
    for sheet_name, df in rason_data.items():
        #Jika hendak mem-filter data untuk per tekanan, gunakan baris kode di bawah ini (hapus tanda # pada baris kode di bawah ini)
        #df = df.loc[(df.index == 0) | (df.pressure.isin([850, 700, 500, 300, 200, 150, 100, 50, 30]))]
        df['depression'] = df['temperature'] - df['dewpoint']
        df = df[['time', 'pressure', 'height', 'temperature', 'dewpoint', 'depression', 'direction', 'speed', 'u_wind', 'v_wind', 'station', 'station_number', 'latitude', 'longitude', 'elevation', 'pw']]
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
# Jangan jalankan sel ini!
# Gunakan ini untuk soal UTS
#with pd.ExcelWriter(r"/content/drive/MyDrive/Test2.xlsx") as writer:
     #df_dates = []
     #for i in rason_data.keys():
          #temp_df = rason_data[i]
          #df_dates.append(temp_df)
     #rason_data['merged'] = pd.concat(df_dates)
     #df= rason_data['merged']
     #df = df.loc[(df.index == 0) | (df.pressure.isin([850, 700, 500, 300, 200, 150, 100, 50, 30]))]
     #df['depression'] = df['temperature'] - df['dewpoint']
     #df = df[['time', 'pressure', 'height', 'temperature', 'dewpoint', 'depression', 'direction', 'speed', 'u_wind', 'v_wind', 'station', 'station_number', 'latitude', 'longitude', 'elevation', 'pw']]
     #df.to_excel(writer, index=False)

In [ ]:
#Membuat dictionary untuk memasukkan dataframe dari data radiosonde 12 jam-an
rason_data = {}

#Mengambil data radiosonde dari library siphon berdasaran waktu dan stasiun
for date in dates:
  print("Memproses data tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
  try:
    rason_data[date.strftime(r'%Y-%m-%d %HZ')] = df = WyomingUpperAir.request_data(date, station_icao)
  except KeyError:
    print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
  except ValueError:
    print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
  except IndexError:
    print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
  except HTTPError:
    raise SystemExit("Server sedang sibuk, jalankan ulang sel ini")
    break

#Mengecek berapa apakah data pengamatan yang tersedia sudah lengkap selama sebulan
if len(rason_data.items()) == len(dates):
  print("\nData pengamatan tersedia lengkap selama sebulan")
else:
  print("\nData pengamatan yang tersedia hanya %d dari %d" % (len(rason_data.items()), len(dates)))

In [ ]:
import time

max_retries = 10  # Set the maximum number of retries
retry_delay = 30 # Set the delay in seconds before retrying

retries = 0
while retries < max_retries:
    try:
        print(f"Attempt {retries + 1} of {max_retries}...")

        #Membuat dictionary untuk memasukkan dataframe dari data radiosonde 12 jam-an
        rason_data = {}

        #Mengambil data radiosonde dari library siphon berdasaran waktu dan stasiun
        for date in dates:
          print("Memproses data tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
          try:
            rason_data[date.strftime(r'%Y-%m-%d %HZ')] = df = WyomingUpperAir.request_data(date, station_icao)
          except KeyError:
            print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
          except ValueError:
            print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
          except IndexError:
            print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
          except HTTPError:
            raise SystemExit("Server sedang sibuk, jalankan ulang sel ini")
            break

        #Mengecek berapa apakah data pengamatan yang tersedia sudah lengkap selama sebulan
        if len(rason_data.items()) == len(dates):
          print("\nData pengamatan tersedia lengkap selama sebulan")
        else:
          print("\nData pengamatan yang tersedia hanya %d dari %d" % (len(rason_data.items()), len(dates)))

        # If the code above runs without error, break the loop
        print("Code executed successfully!")
        break

    except Exception as e:
        print(f"An error occurred: {e}")
        retries += 1
        if retries < max_retries:
            print(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        else:
            print("Maximum retries reached. Could not execute the code.")

In [ ]:
import time

max_retries = 10  # Set the maximum number of retries
retry_delay = 30 # Set the delay in seconds before retrying

retries = 0
while retries < max_retries:
    try:
        print(f"Attempt {retries + 1} of {max_retries}...")

        #Membuat dictionary untuk memasukkan dataframe dari data radiosonde 12 jam-an
        rason_data = {}

        #Mengambil data radiosonde dari library siphon berdasaran waktu dan stasiun
        for date in dates:
          print("Memproses data tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
          try:
            rason_data[date.strftime(r'%Y-%m-%d %HZ')] = df = WyomingUpperAir.request_data(date, station_icao)
          except KeyError:
            print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
          except ValueError:
            print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
          except IndexError:
            print("Data tidak tersedia untuk pengamatan tanggal %s" % (date.strftime(r'%Y-%m-%d %HZ')))
          except HTTPError:
            # Jangan raise SystemExit di sini, biarkan exception ini ditangkap oleh blok luar
            # This allows the outer retry loop to handle the HTTPError
            print("HTTPError: Server sedang sibuk, mencoba lagi...")
            # The `break` here will still exit the inner for loop but the outer
            # while loop will continue if retries < max_retries
            #break # Break the inner for loop to trigger a retry attempt for the whole set of dates


        #Mengecek berapa apakah data pengamatan yang tersedia sudah lengkap selama sebulan
        # This check might not be accurate if an HTTPError occurs and the inner loop breaks.
        # Consider moving this check outside the retry loop or adjust it.
        if len(rason_data.items()) == len(dates):
          print("\nData pengamatan tersedia lengkap selama sebulan")
        else:
          print("\nData pengamatan yang tersedia hanya %d dari %d" % (len(rason_data.items()), len(dates)))
          # If data is not complete, and no HTTPError occurred, you might want to break
          # the retry loop here as well if you don't expect missing data otherwise.
          # Or, you might want to retry in case the missing data was a temporary issue.
          # For now, we assume you want to retry if an HTTPError caused incomplete data.


        # If the code above runs without error AND collected all dates, break the retry loop
        # If an HTTPError caused a break in the inner loop, rason_data might be incomplete,
        # and the retry loop will continue.
        if len(rason_data.items()) == len(dates):
            print("Code executed successfully and all data collected!")
            break # Exit the retry loop if all data is successfully fetched

    except Exception as e:
        # This catches any other exceptions during the process, including
        # the HTTPError if the inner break was removed.
        print(f"An error occurred: {e}")
        retries += 1
        if retries < max_retries:
            print(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        else:
            print("Maximum retries reached. Could not execute the code.")

# After the loop, check if data was successfully retrieved
if len(rason_data.items()) == len(dates):
    print("Data collection completed successfully.")
else:
    print("Data collection was incomplete after multiple retries.")
    # Handle incomplete data as needed, e.g., print which dates are missing
    collected_dates = [datetime.strptime(d_str, r'%Y-%m-%d %HZ') for d_str in rason_data.keys()]
    missing_dates = [d for d in dates if d not in collected_dates]
    print(f"Missing data for dates: {[d.strftime(r'%Y-%m-%d %HZ') for d in missing_dates]}")